# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [62]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
import statsmodels.formula.api as smf
import patsy
import unidecode
import re
import statsmodels.api as sm
from patsy import build_design_matrices
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from patsy import dmatrices


In [39]:
df = pd.read_csv('previsao_de_renda.csv')

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [41]:
df_treino, df_teste = train_test_split(df, test_size=0.25, random_state=42)

print("Tamanho do conjunto de treino:", df_treino.shape)
print("Tamanho do conjunto de teste:", df_teste.shape)

Tamanho do conjunto de treino: (11250, 16)
Tamanho do conjunto de teste: (3750, 16)


In [42]:
df['log_renda'] = np.log(df['renda'])

In [43]:
df_treino, df_teste = train_test_split(df, test_size=0.25, random_state=42)

In [44]:
df['log_renda'] = np.log(df['renda'])

df_dummies = pd.get_dummies(df.drop(columns=['renda']), drop_first=True)

df_treino, df_teste = train_test_split(df_dummies, test_size=0.25, random_state=42)

X_train = df_treino.drop(columns=['log_renda'])
y_train = df_treino['log_renda']
X_test = df_teste.drop(columns=['log_renda'])
y_test = df_teste['log_renda']

imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_scaled, y_train)
    y_pred = ridge.predict(X_test_scaled)
    r2 = r2_score(y_test, y_pred)
    print(f'Alpha = {alpha:.3f} | R² = {r2:.4f}')


Alpha = 0.000 | R² = 0.2402
Alpha = 0.001 | R² = 0.2402
Alpha = 0.005 | R² = 0.2402
Alpha = 0.010 | R² = 0.2402
Alpha = 0.050 | R² = 0.2402
Alpha = 0.100 | R² = 0.2402


In [16]:
O melhor modelo é com qualquer Alpha usado pois o R² permanece no mesmo valor.

SyntaxError: invalid character '²' (U+00B2) (1890375441.py, line 1)

In [45]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
melhor_r2 = -float('inf')
melhor_alpha = None

print("Resultados LASSO:")
for alpha in alphas:
    lasso = Lasso(alpha=alpha, max_iter=10000)
    lasso.fit(X_train_scaled, y_train)
    y_pred = lasso.predict(X_test_scaled)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Alpha = {alpha:.3f} | R² = {r2:.4f}")
    
    if r2 > melhor_r2:
        melhor_r2 = r2
        melhor_alpha = alpha

print(f"\n Melhor alpha para LASSO: {melhor_alpha} | R² = {melhor_r2:.4f}")

Resultados LASSO:


C:\Users\shinoki\anaconda3\Lib\site-packages\sklearn\base.py:1474: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\shinoki\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(


Alpha = 0.000 | R² = 0.2402
Alpha = 0.001 | R² = 0.2410
Alpha = 0.005 | R² = 0.2405
Alpha = 0.010 | R² = 0.2389
Alpha = 0.050 | R² = 0.2071
Alpha = 0.100 | R² = 0.1681

 Melhor alpha para LASSO: 0.001 | R² = 0.2410


C:\Users\shinoki\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.735e+03, tolerance: 7.054e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


In [ ]:
LASSO teve o melhor desempenho.

In [46]:
def clean_col_name(col):
    col = unidecode.unidecode(col)
    col = re.sub(r'\s+', '_', col)
    col = re.sub(r'[^\w]', '', col)
    if re.match(r'^\d', col):
        col = 'var_' + col
    return col

df_treino_renamed = df_treino.rename(columns=clean_col_name)
df_teste_renamed = df_teste.rename(columns=clean_col_name)

print(df_treino_renamed.columns.tolist())

['Unnamed_0', 'index', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'idade', 'tempo_emprego', 'qt_pessoas_residencia', 'mau', 'log_renda', 'data_ref_20150201', 'data_ref_20150301', 'data_ref_20150401', 'data_ref_20150501', 'data_ref_20150601', 'data_ref_20150701', 'data_ref_20150801', 'data_ref_20150901', 'data_ref_20151001', 'data_ref_20151101', 'data_ref_20151201', 'data_ref_20160101', 'data_ref_20160201', 'data_ref_20160301', 'sexo_M', 'tipo_renda_Bolsista', 'tipo_renda_Empresario', 'tipo_renda_Pensionista', 'tipo_renda_Servidor_publico', 'educacao_Pos_graduacao', 'educacao_Secundario', 'educacao_Superior_completo', 'educacao_Superior_incompleto', 'estado_civil_Separado', 'estado_civil_Solteiro', 'estado_civil_Uniao', 'estado_civil_Viuvo', 'tipo_residencia_Casa', 'tipo_residencia_Com_os_pais', 'tipo_residencia_Comunitario', 'tipo_residencia_Estudio', 'tipo_residencia_Governamental']


In [47]:
def stepwise_selection(data, target, signif_level=0.05):
    features = list(data.columns)
    features.remove(target)

    selected_features = []
    current_score = None

    while True:
        changed = False
      
        remaining_features = [f for f in features if f not in selected_features]
        pvals = pd.Series(index=remaining_features, dtype=float)

        for feature in remaining_features:
            formula = f"{target} ~ " + " + ".join(selected_features + [feature])
            model = smf.ols(formula=formula, data=data).fit()

            pval = None
            for name in model.pvalues.index:
                if name == feature or name.startswith(feature + "["):
                    pval = model.pvalues[name]
                    break
            if pval is None:
                pval = 1
            pvals[feature] = pval

        if not pvals.empty:
            min_pval = pvals.min()
            if min_pval < signif_level:
                best_feature = pvals.idxmin()
                selected_features.append(best_feature)
                changed = True

        if selected_features:
            formula = f"{target} ~ " + " + ".join(selected_features)
            model = smf.ols(formula=formula, data=data).fit()
            pvalues = model.pvalues.iloc[1:] 
            max_pval = pvalues.max()
            if max_pval > signif_level:
                worst_feature = pvalues.idxmax()
                selected_features.remove(worst_feature)
                changed = True

        if not changed:
            break

    formula_final = f"{target} ~ " + " + ".join(selected_features)
    model_final = smf.ols(formula=formula_final, data=data).fit()
    return model_final, formula_final



In [55]:
modelo_stepwise, formula_stepwise = stepwise_selection(df_treino_renamed, target='log_renda')

y_train, X_train = dmatrices(formula_stepwise, data=df_treino_renamed, return_type='dataframe')
design_info = X_train.design_info 

modelo_stepwise = sm.OLS(y_train, X_train).fit()

y_test, X_test = dmatrices(formula_stepwise, data=df_teste_renamed, return_type='dataframe')

y_pred = modelo_stepwise.predict(X_test)

r2 = r2_score(y_test, y_pred)
print(f"R² na base de teste: {r2:.4f}")


R² na base de teste: 0.2498


In [ ]:
O melhor modelo é o Stepwise

In [ ]:
Se considerarmos apenas o desempenho (R²): Stepwise vence com 0.2498, mas sem regularização. Se buscarmos robustez, bom desempenho, simplicidade:
LASSO (α=0.001) é o mais equilibrado. Tem desempenho quase igual ao Stepwise, faz seleção automática de variáveis, é regularizado, então menos sujeito a overfitting.



In [61]:
df_treino_renamed['tempo_emprego'].fillna(df_treino_renamed['tempo_emprego'].mean(), inplace=True)
df_treino_renamed['log_idade'].fillna(df_treino_renamed['log_idade'].mean(), inplace=True)

df_teste_renamed['tempo_emprego'].fillna(df_teste_renamed['tempo_emprego'].mean(), inplace=True)
df_teste_renamed['log_idade'].fillna(df_teste_renamed['log_idade'].mean(), inplace=True)

scaler = StandardScaler()
df_treino_renamed[colunas_numericas] = scaler.fit_transform(df_treino_renamed[colunas_numericas])
df_teste_renamed[colunas_numericas] = scaler.transform(df_teste_renamed[colunas_numericas])

pca = PCA(n_components=5)
X_pca = pca.fit_transform(df_treino_renamed[colunas_numericas])
X_test_pca = pca.transform(df_teste_renamed[colunas_numericas])

scaler = StandardScaler()
colunas_numericas = df_treino_renamed.select_dtypes(include='number').columns.drop('log_renda')

df_treino_renamed[colunas_numericas] = scaler.fit_transform(df_treino_renamed[colunas_numericas])
df_teste_renamed[colunas_numericas] = scaler.transform(df_teste_renamed[colunas_numericas])

pca = PCA(n_components=5)
X_pca = pca.fit_transform(df_treino_renamed[colunas_numericas])
X_test_pca = pca.transform(df_teste_renamed[colunas_numericas])

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train.values.ravel())

y_pred_rf = rf.predict(X_test)
print("R² Random Forest:", r2_score(y_test, y_pred_rf))



C:\Users\shinoki\AppData\Local\Temp\ipykernel_16816\3243572487.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_treino_renamed['tempo_emprego'].fillna(df_treino_renamed['tempo_emprego'].mean(), inplace=True)
C:\Users\shinoki\AppData\Local\Temp\ipykernel_16816\3243572487.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we

R² Random Forest: 0.303847566663369


In [63]:
features = ['idade', 'qtd_filhos', 'tempo_emprego', 'qt_pessoas_residencia', 'idade2', 'log_idade']
target = 'log_renda' 

X_train = df_treino_renamed[features]
y_train = df_treino_renamed[target]

X_test = df_teste_renamed[features]
y_test = df_teste_renamed[target]

arvore = DecisionTreeRegressor(random_state=42)
arvore.fit(X_train, y_train)

y_pred = arvore.predict(X_test)

r2 = r2_score(y_test, y_pred)
print(f"R² da árvore de regressão na base de teste: {r2:.4f}")


R² da árvore de regressão na base de teste: 0.0114
